In [3]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [4]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-persist-{username}", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.large"
)
s.submit(idempotent=True)

Waiting for server node-persist-am14305_nyu_edu's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-persist-am14305_nyu_edu
Id,a5ab3605-76e8-44ad-a835-ea4165404c05
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.3.169 (v4) Type: fixed MAC: fa:16:3e:eb:3e:82
Network Name,sharednet1
Created At,2025-04-27T14:47:14Z
Keypair,am14305_nyu_edu-jupyter
Reservation Id,None
Host Id,58ca395f1941a9db0b9e19ed3517dabef1db4423a821e7b9804af3b4


In [5]:
s.associate_floating_ip()

In [6]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.26.153 port 22.


Connection successful


In [7]:
s.refresh()
s.show(type="widget")

Attribute,node-persist-am14305_nyu_edu
Id,a5ab3605-76e8-44ad-a835-ea4165404c05
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.large
Addresses,sharednet1: IP: 10.56.3.169 (v4) Type: fixed MAC: fa:16:3e:eb:3e:82 IP: 129.114.26.153 (v4) Type: floating MAC: fa:16:3e:eb:3e:82
Network Name,sharednet1
Created At,2025-04-27T14:47:14Z
Keypair,am14305_nyu_edu-jupyter
Reservation Id,None
Host Id,58ca395f1941a9db0b9e19ed3517dabef1db4423a821e7b9804af3b4


In [8]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

In [9]:
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-8000', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


In [10]:
s.execute("git clone https://github.com/Jex2l/NYC-Taxi-Demand-Prediction-with-Weather-and-Flight-Data-Integration.git")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.26.153: b'305a6eb07b717878fe8009250afceb83'
  warnings.warn(
Cloning into 'NYC-Taxi-Demand-Prediction-with-Weather-and-Flight-Data-Integration'...


<Result cmd='git clone https://github.com/Jex2l/NYC-Taxi-Demand-Prediction-with-Weather-and-Flight-Data-Integration.git' exited=0>

In [11]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null

Running kernel seems to be up-to-date.

Restarting services...
 systemctl restart packagekit.service

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>